# AR-SSL4M Pretraining on Google Colab

This notebook handles the setup and pretraining of the AR-SSL4M model using data from Google Drive.

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check dataset path
import os
dataset_path = '/content/drive/MyDrive/dataset/LIDC-IDRI'
print(os.listdir('/content/drive/MyDrive/dataset/LIDC-IDRI'))
if os.path.exists(dataset_path):
    print(f"Dataset found at {dataset_path}")
    print(os.listdir(dataset_path))
else:
    print(f"Dataset NOT found at {dataset_path}. Please check your Drive structure.")

['LIDC-IDRI', 'AR-SSL4M-DEMO', 'patch_random_spatial', 'Untitled folder', 'colab_train_list.txt', 'output']
Dataset found at /content/drive/MyDrive/dataset/LIDC-IDRI
['LIDC-IDRI', 'AR-SSL4M-DEMO', 'patch_random_spatial', 'Untitled folder', 'colab_train_list.txt', 'output']


In [4]:
# Data Verification and Cleaning
# This cell checks all .npy files in the dataset to ensure they are valid and not empty.
# It will regenerate the 'colab_train_list.txt' excluding any corrupted files.

import os
import numpy as np
from tqdm import tqdm

drive_dataset_path = '/content/drive/MyDrive/dataset/LIDC-IDRI'
patch_dir = os.path.join(drive_dataset_path, 'patch_random_spatial')
list_file_path = os.path.join(drive_dataset_path, 'colab_train_list.txt')

valid_files = []
corrupted_files = []

if os.path.exists(patch_dir):
    npy_files = [f for f in os.listdir(patch_dir) if f.endswith('.npy')]
    print(f"Checking {len(npy_files)} files in {patch_dir}...")

    for f in tqdm(npy_files):
        full_path = os.path.join(patch_dir, f)
        try:
            # Try loading the file
            data = np.load(full_path, mmap_mode='r') # Use mmap_mode='r' for faster checking without full read
            # Check shape/size if necessary, e.g.
            if data.size == 0 or data.shape != (128, 128, 128):
                 # Double check by fully loading if mmap is unsure or for strict size check
                 data = np.load(full_path)
                 if data.size == 0:
                    print(f"Skipping empty file: {f}")
                    corrupted_files.append(full_path)
                    continue

            valid_files.append(full_path)
        except Exception as e:
            print(f"Error reading {f}: {e}")
            corrupted_files.append(full_path)

    # Update the list file with only valid paths
    with open(list_file_path, 'w') as f:
        f.write('\n'.join(valid_files))

    print(f"\nVerification complete.")
    print(f"Valid files: {len(valid_files)}")
    print(f"Corrupted/Empty files removed: {len(corrupted_files)}")
    print(f"Updated training list at: {list_file_path}")

else:
    print(f"Patch directory not found: {patch_dir}")

Checking 24850 files in /content/drive/MyDrive/dataset/LIDC-IDRI/patch_random_spatial...


100%|██████████| 24850/24850 [4:35:15<00:00,  1.50it/s]


Verification complete.
Valid files: 24850
Corrupted/Empty files removed: 0
Updated training list at: /content/drive/MyDrive/dataset/LIDC-IDRI/colab_train_list.txt


In [3]:
# Clone the repository (if not already present)
# Cloning from your GitHub repository as requested
!git clone https://github.com/tanglehunter00/AR-SSL4M-DEMO.git

# IMPORTANT: If you are running this notebook and the code is NOT on Drive,
# you need to upload the code files to Colab runtime.

project_root = '/content/AR-SSL4M-DEMO'
import os
if os.path.exists(project_root):
    %cd {project_root}
else:
    print("Project root not found. Please clone or upload your code.")

Cloning into 'AR-SSL4M-DEMO'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 290 (delta 14), reused 9 (delta 9), pack-reused 274 (from 1)
Receiving objects: 100% (290/290), 1.39 MiB | 27.36 MiB/s, done.
Resolving deltas: 100% (134/134), done.
/content/AR-SSL4M-DEMO


In [4]:
# Install dependencies
!pip install timm monai transformers fire

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 11.7 MB/s eta 0:00:00


In [5]:
# Update dataset configuration paths dynamically
# We need to point the dataset config to the list files in Google Drive

# Assuming your list files are also in the dataset folder on Drive
# You might need to generate these list files if they contain absolute local paths from your PC.
# Here we create a new list file based on the Drive path.

import os

drive_dataset_path = '/content/drive/MyDrive/dataset/LIDC-IDRI'
patch_dir = os.path.join(drive_dataset_path, 'patch_random_spatial')
list_file_path = os.path.join(drive_dataset_path, 'colab_train_list.txt')

if os.path.exists(patch_dir):
    npy_files = [f for f in os.listdir(patch_dir) if f.endswith('.npy')]
    full_paths = [os.path.join(patch_dir, f) for f in npy_files]

    with open(list_file_path, 'w') as f:
        f.write('\n'.join(full_paths))
    print(f"Created training list at {list_file_path} with {len(full_paths)} files.")
else:
    print("Patch directory not found. Please ensure 'patch_random_spatial' exists inside 'dataset/demo'.")


Created training list at /content/drive/MyDrive/dataset/LIDC-IDRI/colab_train_list.txt with 24850 files.


In [6]:
# Modify configs/datasets.py to use the Colab path
# We will do this by writing a temporary config file or modifying the file in place if possible.
# A safer way is to rely on the fact that we can update the config dynamically,
# but the current codebase reads from a file.
# Let's modify pretrain/configs/datasets.py directly.

config_path = 'pretrain/configs/datasets.py'

new_config_content = f"""
from dataclasses import dataclass

@dataclass
class custom_dataset:
    dataset: str = "custom_dataset"
    file: str = "image_dataset.py"
    train_split: str = "train"
    test_split: str = "validation"
    # Pointing to the generated list file on Drive
    spatial_path: str = "{list_file_path}"
    contrast_path: str = "{list_file_path}"
    semantic_path: str = "{list_file_path}"
    img_size = [128, 128, 128]
    patch_size = [16, 16, 16]
    attention_type = 'prefix'
    add_series_data = False
    add_spatial_data = True
    is_subset = False
    series_length = 4
"""

with open(config_path, 'w') as f:
    f.write(new_config_content)

print("Updated datasets.py configuration.")

Updated datasets.py configuration.


In [9]:
# Run Pretraining
# Set batch size to 32 as requested
# Using newModel.py (ensure you imported ReconModel from newModel in main.py if that was the intent,
# OR ensure model.py is updated with your changes)

# Note: The user requested to use 'newModel.py'.
# You might need to rename newModel.py to model.py OR modify main.py to import from newModel.
# Here we assume main.py still imports from model.py.
# We will rename newModel.py to model.py for this run to ensure the new logic is used.

!cp pretrain/newModel.py pretrain/model.py

# Switch to pretrain directory
%cd pretrain

!mkdir -p /content/drive/MyDrive/dataset/LIDC-IDRI/output

# Run training
!python main.py \
    --enable_fsdp False \
    --output_dir /content/drive/MyDrive/dataset/demo/output \
    --batch_size_training 128 \
    --num_epochs 5 \
    --save_metrics True \
    --num_workers_dataloader 4

cp: cannot stat 'pretrain/newModel.py': No such file or directory
[Errno 2] No such file or directory: 'pretrain'
/content/AR-SSL4M-DEMO/pretrain
2026-01-26 21:45:56.481968: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-26 21:45:56.500074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769463956.522029    4891 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769463956.528501    4891 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been register